In [1]:
import odbc
import win32com.client
import pandas as pd
import pyodbc
import datetime

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [2]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)
cnxn.close()

In [4]:
def check_excel(file,excel_id,index):
    global output
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.EnableEvents = False
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)
            
        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)
                    try:
                        for o in x.Ranges(1).QueryTable.Parameters:


                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          o.SourceRange.Address,
                                          o.SourceRange.Parent.Name,
                                          o.SourceRange.Value,
                                          excel_id,
                                          index])
                        flag = 1
                    except:
                        flag = 0
                    if flag == 0:

                        try:
                            if len(i.ListObject.QueryTable.Parameters) != 0:

                                for p in i.ListObject.QueryTable.Parameters:

                                    datos.append([x,
                                                  x.ODBCConnection.CommandText,
                                                  x.ODBCConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value,
                                                  excel_id,
                                                  index])
                            else:

                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name, #EDIT None
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])

                        except Exception as e:
                            try:
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])
                            except:
                                try:
                                    if len(i.ListObject.QueryTable.Parameters) != 0:

                                        for p in i.ListObject.QueryTable.Parameters:
                                            datos.append([x,
                                                          x.OLEDBConnection.CommandText,
                                                          x.OLEDBConnection.Connection,
                                                          i.Parent.Name,
                                                          celda,
                                                          coordenada,
                                                          p.SourceRange.Address,
                                                          p.SourceRange.Parent.Name,
                                                          p.SourceRange.Value,
                                                          excel_id,
                                                          index])
                                    else:
                                        datos.append([x,
                                                      x.ODBCConnection.CommandText,
                                                      x.ODBCConnection.Connection,
                                                      i.Parent.Name, #EDIT None
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])

                                except:
                                    try:
                                        datos.append([x,
                                                      x.OLEDBConnection.CommandText,
                                                      x.OLEDBConnection.Connection,
                                                      i.Parent.Name,
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])
                                    except:                            
                                        datos.append([x,
                                                  None,
                                                  None,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  None,
                                                  None,
                                                  None,
                                                  excel_id,
                                                  index])

            

        except Exception as c:
            return c , []


        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None'])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
    
        if type(datos[0]) == str:
            
            output +=[datos]
        else:
            output += datos

        
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0
    except:
        return 'No se pudo abrir .xlsx',[]

In [11]:
for i in range(201):
    if int(i/100) == i/100:
        print(i)

0
100
200


In [47]:
output = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    if int(index/1000) == index/1000:
        print(index)
#        if index > 0 :
#            raise
    archivo = row['path']+row['filename']
    #matar proceso excel
    result = check_excel(archivo,row['excel_id'],index)
    #result = check_excel('C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx')
    if result != 0:
        
        errores.append([row['excel_id'],archivo,result,index])
            
    else:
        pass
    #raise 
    #if index == 5:
        
end = datetime.datetime.today()
print(end-start)
raise
#     respuesta = result[0]
#     result[1]

0
1000
2000


KeyboardInterrupt: 

In [446]:
print(df.loc[2]['path']+df.loc[2]['filename'])

Z:\....Candidatos a borrar\Archivos Viejos\TODO\IMPRESION ROTULOS DEPOSITO.xls


In [49]:
len(output)

9452

In [39]:
df.loc[2]

excel_id                                                               42
path                     Z:\....Candidatos a borrar\Archivos Viejos\TODO\
filename                                   IMPRESION ROTULOS DEPOSITO.xls
date_of_insertion                              2020-01-23 12:10:12.707000
start_processing_date                                                 NaT
end_processing_date                                                  None
process_id                                                           None
error_msg                                                            None
Name: 2, dtype: object

In [132]:
dfParametros.to_excel("relevamiento.xlsx")

In [ ]:
df[df['excel_id']==70]

In [67]:
print(df.loc[30]['path']+df.loc[30]['filename'])

Z:\Consultas\Articulos\Copia de stocks y demanda próximos 5 días.xlsx


In [64]:
dfParametros[dfParametros['excel_id']=='70'] # <------------------------

,name_excel_connection,sql_excel_connection,str_conn_excel_connection,sheet_excel_connection_location,Inicio_tabla_formula,cell_excel_connection_location,cell_excel_connection_location_parameter,sheet_connection_location_parameter,content_excel_connection_location_parameter,excel_id,index
55,Consulta desde ESTIMACIONES,"select a.codigo, ar.descrip, a.pt, a.pro, a.pr...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Hoja1,codigo,$A$1,None,None,None,70,30


In [56]:
dfParametros # <------------------------

,name_excel_connection,sql_excel_connection,str_conn_excel_connection,sheet_excel_connection_location,Inicio_tabla_formula,cell_excel_connection_location,cell_excel_connection_location_parameter,sheet_connection_location_parameter,content_excel_connection_location_parameter,excel_id,index
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Lista Tipos y Colores,codigo,$A$1,None,None,None,40,0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Lista Tipos y Colores,codigo,$D$1,None,None,None,40,0
2,None,None,None,None,None,None,None,None,None,None,None
3,Consulta desde ESTIMACIONES,"SELECT f.articulo, f.insumo, f.color, i.descri...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,BOLSAS,articulo,$A$1,None,None,None,42,2
4,Consulta desde ESTIMACIONES1,"SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PLANCHAR,ARTICULO,$A$4,None,None,None,42,2
...,...,...,...,...,...,...,...,...,...,...,...
9447,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$17,Posiciones diferencias x 20,None,2208,2168
9448,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$18,Posiciones diferencias x 20,None,2208,2168
9449,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$19,Posiciones diferencias x 20,None,2208,2168
9450,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$20,Posiciones diferencias x 20,None,2208,2168


In [53]:
len(errores)+len(output)

9503

In [412]:
dfParametros[dfParametros.isnull().any(1)]

,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,source_range_address,source_range_parent_name,source_range_value,excel_id,index


In [83]:
len(dfParametros), len(dfParametros.dropna())

(9452, 9452)

In [91]:
dfParametros = dfParametros[dfParametros['excel_id'] != 'None']

In [94]:
dfParametros['excel_id'] = dfParametros['excel_id'].astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None


In [415]:
resultado_df = pd.merge(df,dfParametros,on='excel_id',how='right')

In [416]:
len(dfParametros),len(resultado_df)

(6935, 6935)

In [418]:
resultado_df.to_excel('output_re1.xlsx')

In [393]:
resultado_df.tail()

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,10
16732,1410.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Conexión,"SELECT vida.articulo, vida.cp, vida.estado_cod...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,X cp,articulo,$A$5,$B$2,X cp,7355.0
16733,00693313008,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,7376.0
16734,29014502002,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,7377.0
16735,91203454007,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde ESTIMACIONES1,"select codigo, cantidad, frente, columna, nive...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Historial articulo,codigo,$A$4,$B$2,Historial articulo,7379.0
16736,91203454007,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde ESTIMACIONES2,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Historial articulo,codigo,$H$4,$B$2,Historial articulo,7379.0


In [373]:
dfParametros[dfParametros['excel_id'] == 3]

,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,excel_id,10


,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,excel_id
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,codigo,None,None,None,40,0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,codigo,None,None,None,40,0
2,None,None,None,None,None,None,None,None,41
3,Conexión,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TALLERES,codigo,None,None,None,42
4,Conexión1,"SELECT tipos.codigo, tipos.nombre, tipos.categ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TIPOS,codigo,None,None,None,42
...,...,...,...,...,...,...,...,...,...
104,None,None,None,None,None,None,None,None,79
105,Conexión,"SELECT caj_pedidos_por_dia.fecha, Sum(caj_pedi...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Pedidos por Día,2009-03-20 00:00:00+00:00,None,None,None,80
106,Conexión1,"SELECT caj_pedidos_por_dia.zona, caj_pedidos_p...",ODBC;DSN=ESTIMACIONES;Description=f;UID=estima...,Detalle del Día,0011,None,None,None,80
107,Conexión2,"SELECT caj_pedidos_por_dia.zona, caj_pedidos_p...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Detalle entre Fechas,2008-06-05 00:00:00+00:00,None,None,None,80


In [44]:
dfParametros['Texto de comando'][3]

'select * from estimaciones..movstock with (nolock) where fecha_pura = ? and movimiento = ?'

In [316]:
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
file = df.loc[40]['path']+df.loc[40]['filename']
wb = xl.Workbooks.Open(file, None, True)

datos = []

for x in wb.Connections:
        
    

    for i in x.Ranges:
        try:
            #PRIMER DATO QUE TRAE LA PRIMER CELDA
            celda = i.Cells(1,1)
            #LA COORDENADA DE LA PRIMER CELDA
            coordenada = i.Cells(1,1).Address
        except Exception as p:
            print(p)
        try:
            for o in x.Ranges(1).QueryTable.Parameters:
                
                
                datos.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              celda,
                              coordenada,
                              o.SourceRange.Address,
                              o.SourceRange.Parent.Name,
                              o.SourceRange.Value])
            flag = 1
        except:
            flag = 0
        if flag == 0:
            
            try:
                if len(i.ListObject.QueryTable.Parameters) != 0:

                    for p in i.ListObject.QueryTable.Parameters:

                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      coordenada,
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])
                else:

                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  None,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])

            except Exception as e:
                try:
                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  i.Parent.Name,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])
                except:
                    try:
                        if len(i.ListObject.QueryTable.Parameters) != 0:

                            for p in i.ListObject.QueryTable.Parameters:
                                datos.append([x,
                                              x.OLEDBConnection.CommandText,
                                              x.OLEDBConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              p.SourceRange.Address,
                                              p.SourceRange.Parent.Name,
                                              p.SourceRange.Value])
                        else:
                            datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  None,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])

                    except:
                        try:
                            datos.append([x,
                                          x.OLEDBConnection.CommandText,
                                          x.OLEDBConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          None,
                                          None,
                                          None])
                        except:                            
                            datos.append([x,
                                      None,
                                      None,
                                      i.Parent.Name,
                                      celda,
                                      coordenada,
                                      None,
                                      None,
                                      None])


        
        
    
    
    
    

'Detalle De Zona'

In [317]:
if type(datos[0]) == list:
    datos = [list(map(str,i))for i in datos]

else:
    datos = list(map(str,datos))

In [335]:
pd.DataFrame(datos)

,0,1,2,3,4,5,6,7,8,9,10
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$A$1,None,None,None,40,0.0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$D$1,None,None,None,40,0.0
2,None,None,None,None,None,None,None,None,41,1,NaN
3,Consulta desde ESTIMACIONES,"SELECT f.articulo, f.insumo, f.color, i.descri...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,articulo,$A$1,None,None,None,42,2.0
4,Consulta desde ESTIMACIONES1,"SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$4,None,None,None,42,2.0
5,Consulta desde estimaciones2,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,codigo,$H$3,None,None,None,42,2.0
6,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,39153736009,42,2.0
7,Consulta desde estimaciones4,SELECT DISTINCT ARTICULO FROM VIDA WHERE ESTAD...,ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$5,None,None,None,42,2.0
8,None,None,None,None,None,None,None,None,43,3,NaN
9,Conexión,"SELECT faltannu.cp, faltannu.pedido, faltannu....",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Faltante entre Fechas,cp,$A$8,$B$4,Faltante entre Fechas,2015-08-24 00:00:00+00:00,44,4.0


In [ ]:
# Lista de posibles erroes en excel al abrir (error)
#Z:\Consultas\Estimaciones\VIEJO\Posibles Sobrantes para Gondola.xlsx
#Z:\Consultas\Estimaciones\Proyección de pedidos-unidades sobre zonas ingresadas 2 - no funciona 20160215.xlsx
#Z:\Consultas\Mano de Obra\Actualizador de Mano de Obra.xlsx

In [12]:
import pandas as pd

In [20]:
output = pd.read_excel("temp_output.xlsx")

In [19]:
output

""


In [15]:
errores = pd.read_excel("temp_errores.xlsx")
errores

,Unnamed: 0,0,1,2,3
0,0,17351,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"('No se pudo abrir .xlsx', [])",0
1,1,17352,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"('No se pudo abrir .xlsx', [])",1
2,2,17353,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"('No se pudo abrir .xlsx', [])",2
3,3,17354,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"('No se pudo abrir .xlsx', [])",3
4,4,17355,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"('No se pudo abrir .xlsx', [])",4
...,...,...,...,...,...
17496,17496,23845,Z:\LogisticaDelis\Ruteos\Presupuesto\Presupues...,"(AttributeError(""'NoneType' object has no attr...",17496
17497,17497,23846,Z:\LogisticaDelis\Ruteos\Presupuesto\Presupues...,"(AttributeError(""'NoneType' object has no attr...",17497
17498,17498,23847,Z:\LogisticaDelis\Ruteos\Presupuesto\Presupues...,"(AttributeError(""'NoneType' object has no attr...",17498
17499,17499,23848,Z:\LogisticaDelis\Ruteos\Presupuesto\Presupues...,"(AttributeError(""'NoneType' object has no attr...",17499


In [37]:
len(errores)

17501

In [17]:
len(output)

17501

In [33]:
errores[errores[0]==23849]

,Unnamed: 0,0,1,2,3
17500,17500,23849,Z:\LogisticaDelis\Ruteos\Presupuesto\Presupues...,"(AttributeError(""'NoneType' object has no attr...",17500


In [36]:
errores[2].unique()

array(["('No se pudo abrir .xlsx', [])",
       "(com_error(-2147417851, 'El servidor lanzó una excepción.', None, None), [])",
       '(AttributeError("\'NoneType\' object has no attribute \'Connections\'"), [])'],
      dtype=object)

In [37]:
errores = pd.read_excel("temp_errores.xlsx")

In [42]:
errores[1][0]

'Z:\\Desarrollo_Ventas\\HOJA DE RUTA FERREYRA\\IVAN NAONE\\TALLER - IVAN NAONE 26-1.xlsx'

In [1]:
2+2

4

In [1]:
import pandas as pd

In [50]:
excel1 = pd.read_excel("temp_output.xlsx")

In [53]:
excel1.iloc[-1]

Unnamed: 0                                                28211
0                                                    Conexión58
1             SELECT pcp_op_view4.op, pcp_op_view4.cod8, pcp...
2             ODBC;DSN=eSTIMACIONES;Description=ESTIMACIONES...
3                                                         64679
4                                                            op
5                                                         $O$52
6                                                         $A$48
7                                                         64679
8                                                       64679.0
9                                                         33735
10                                                        12600
11                                   2020-03-13 04:21:20.601000
12                                   2020-03-13 04:21:24.087000
Name: 28211, dtype: object

In [33]:
len(excel1)

4669

In [36]:
len(excel1)

28212

In [39]:
len(errores) + len(excel1)

32968

In [34]:
4669 - 3968

701

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12
437,437,Base_Morosidad,Base_Morosidad_Alternativa,OLEDB;Provider=Microsoft.ACE.OLEDB.12.0;User I...,Gerente Division,None,$A$10,None,None,None,19956,437,2020-03-12 09:57:43.280,2020-03-12 09:57:47.565
461,461,Base_Morosidad,Base_Morosidad,OLEDB;Provider=Microsoft.ACE.OLEDB.12.0;User I...,Detalle_Campañas,Div,$B$27,None,None,None,19981,462,2020-03-12 09:59:58.270,2020-03-12 10:00:09.723
462,462,Base_Morosidad,Base_Morosidad,OLEDB;Provider=Microsoft.ACE.OLEDB.12.0;User I...,Gerente Division,None,$A$10,None,None,None,19981,462,2020-03-12 09:59:58.270,2020-03-12 10:00:09.779
463,463,Base_Morosidad,Base_Morosidad,OLEDB;Provider=Microsoft.ACE.OLEDB.12.0;User I...,Gerentes_ delis,Region,$A$8,None,None,None,19981,462,2020-03-12 09:59:58.270,2020-03-12 10:00:09.838
464,464,Base_Morosidad,Base_Morosidad,OLEDB;Provider=Microsoft.ACE.OLEDB.12.0;User I...,Líderes,Region,$A$9,None,None,None,19981,462,2020-03-12 09:59:58.270,2020-03-12 10:00:09.891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28207,28207,Conexión54,"SELECT pcp_op_view4.op, pcp_op_view4.cod8, pcp...",ODBC;DSN=eSTIMACIONES;Description=ESTIMACIONES...,64675,op,$O$52,$A$48,64675,64675.0,33735,12600,2020-03-13 04:21:20.601,2020-03-13 04:21:23.868
28208,28208,Conexión55,"SELECT pcp_op_view4.op, pcp_op_view4.cod8, pcp...",ODBC;DSN=eSTIMACIONES;Description=ESTIMACIONES...,64676,op,$O$52,$A$48,64676,64676.0,33735,12600,2020-03-13 04:21:20.601,2020-03-13 04:21:23.930
28209,28209,Conexión56,"SELECT pcp_op_view4.op, pcp_op_view4.cod8, pcp...",ODBC;DSN=eSTIMACIONES;Description=ESTIMACIONES...,64677,op,$O$52,$A$48,64677,64677.0,33735,12600,2020-03-13 04:21:20.601,2020-03-13 04:21:23.977
28210,28210,Conexión57,"SELECT pcp_op_view4.op, pcp_op_view4.cod8, pcp...",ODBC;DSN=eSTIMACIONES;Description=ESTIMACIONES...,64678,op,$O$52,$A$48,64678,64678.0,33735,12600,2020-03-13 04:21:20.601,2020-03-13 04:21:24.024


In [54]:
import odbc
import win32com.client
import pandas as pd
import pyodbc
import pickle
import datetime

In [55]:
def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [56]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel with(nolock) where end_processing_date is null'

df = pd.read_sql(query,cnxn)
cnxn.close()

#Vamos a levantar los errores anteriores y vamos a relevar los excels_id que no estan en la lista de errores.

errores = pd.read_excel('full_errores.xlsx')

#Buscamos los excel_id que no se encuentran en errores
df = df[~df['excel_id'].isin(errores[0])].reset_index(drop=True)



In [57]:
len(df)

19146

In [61]:
df.iloc[12600]

excel_id                                                          30876
path                     Z:\Planeamiento\Modelo Semanal\Reprogramacion\
filename                      Comparación de Versiones 12152-12250.xlsm
date_of_insertion                            2020-01-23 13:11:13.923000
start_processing_date                                               NaT
end_processing_date                                                None
process_id                                                         None
error_msg                                                          None
Name: 12600, dtype: object

In [63]:
excel2 = pd.read_excel("temp_errores_13-3.xlsx")

In [66]:
excel2[3].unique().tolist()

[452,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 482,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 510,
 511,
 512,
 513,
 514,
 658,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671,
 672,
 673,
 674,
 675,
 676,
 677,
 678,
 679,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962

In [79]:
try:
    f = open("revivir.txt","r")
    REVIVIR = f.read()
    REVIVIR = int(REVIVIR)
    f.close()
except:
    REVIVIR = 0

In [80]:
REVIVIR

12600

In [72]:
REVIVIR = int(REVIVIR)

In [78]:
df.loc[REVIVIR:]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,33616,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 150.xlsx,2020-01-23 13:12:28.000,NaT,None,None,None
1,33617,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 151.xlsx,2020-01-23 13:12:28.007,NaT,None,None,None
2,33618,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 152.xlsx,2020-01-23 13:12:28.013,NaT,None,None,None
3,33619,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 153.xlsx,2020-01-23 13:12:28.023,NaT,None,None,None
4,33620,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 154.xlsx,2020-01-23 13:12:28.030,NaT,None,None,None
...,...,...,...,...,...,...,...,...
19141,33611,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 145.xlsx,2020-01-23 13:12:27.957,NaT,None,None,None
19142,33612,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 146.xlsx,2020-01-23 13:12:27.963,NaT,None,None,None
19143,33613,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 147.xlsx,2020-01-23 13:12:27.973,NaT,None,None,None
19144,33614,Z:\Produccion\Corte\Mejoramiento Tizadas\,Copia de Calculo Cortes con Historial 148.xlsx,2020-01-23 13:12:27.980,NaT,None,None,None


In [73]:
import os.path, time
hora_creacion = time.ctime(os.path.getmtime("temp_errores_13-3.xlsx"))


In [74]:
hora =  hora_creacion.split(" ")[3].split(":")[0]

In [75]:
hora_actual =  int(time.localtime().tm_hour)

In [76]:
corriendo =int(hora_actual) - int(hora)

In [77]:
corriendo

7

In [2]:
import pandas as pd
df_output = pd.read_excel("temp_output.xlsx")

In [4]:
df_output.iloc[-1][10]

19100

In [3]:
df_errores = pd.read_excel("temp_errores.xlsx")

In [5]:
df_errores.iloc[-1][3]

19057

In [ ]:
#12000

In [85]:
import os
import time
def last_modified(file):
    hora_creacion = time.ctime(os.path.getmtime(file))
    hora =  hora_creacion.split(" ")[3].split(":")[0]
    hora_actual =  int(time.localtime().tm_hour)
    corriendo =int(hora_actual) - int(hora)
    return corriendo

In [83]:
last_modified("temp_errores_13-3.xlsx") 

8

In [86]:
while(True):
    
    if last_modified("temp_errores_13-3.xlsx") >= 1:
        df_output = pd.read_excel("temp_output.xlsx")
        ultimo_output = df_output.iloc[-1][10]
        df_errores = pd.read_excel("temp_errores.xlsx")
        ultimo_errores = df_errores.iloc[-1][3]
        
        if ultimo_output > ultimo_errores:
            print("indice output es mayor")
            nuevo_indice = ultimo_output
        else:
            print("indice errores es mayor")
            nuevo_indice = ultimo_errores
        break
nuevo_indice

FileNotFoundError: [Errno 2] No such file or directory: 'temp_output.xlsx'

In [133]:
int(nuevo_indice)

16200

In [142]:
import email
import smtplib

msg = email.message_from_string('testeo de mail')
msg['From'] = "josesindler@hotmail.com"
msg['To'] = "jhernandez@juanabonita.com"
msg['Subject'] = "helOoooOo"

s = smtplib.SMTP("smtp.live.com",587)
s.ehlo() # Hostname to send for this command defaults to the fully qualified domain name of the local host.
s.starttls() #Puts connection to SMTP server in TLS mode
s.ehlo()
s.login('josesindler@hotmail.com', '******')

s.sendmail("josesindler@hotmail.com", "jhernandez@juanabonita.com", msg.as_string())

s.quit()

(221, b'2.0.0 Service closing transmission channel')

In [87]:
import datetime
import subprocess
import os
import signal 
import pickle

cmd = 'powershell "gps | where {$_.MainWindowTitle } | select Description, id'
  
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)

for line in proc.stdout:
    print(line)

b'\r\n'
b'Description                                                          Id\r\n'
b'-----------                                                          --\r\n'
b'Application Frame Host                                             4656\r\n'
b'Google Chrome                                                      6232\r\n'
b'Microsoft Edge                                                     9904\r\n'
b'Microsoft Edge Content Process                                     1612\r\n'
b'Python                                                             8248\r\n'
b'Thunderbird                                                        8328\r\n'
b'WindowsInternal.ComposableShell.Experiences.TextInput.InputApp.exe 7592\r\n'
b'\r\n'
b'\r\n'


In [83]:
import psutil
 
# Iterate over all running process
for proc in psutil.process_iter():
    try:
        # Get process name & pid from process object.
        processName = proc.name()
        processID = proc.pid
        print(processName , ' ::: ', processID)
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

System Idle Process  :::  0
System  :::  4
Registry  :::  84
RuntimeBroker.exe  :::  164
smss.exe  :::  320
RuntimeBroker.exe  :::  348
svchost.exe  :::  352
browser_broker.exe  :::  372
dwm.exe  :::  408
csrss.exe  :::  432
wininit.exe  :::  540
csrss.exe  :::  548
services.exe  :::  608
lsass.exe  :::  616
svchost.exe  :::  688
svchost.exe  :::  716
svchost.exe  :::  744
fontdrvhost.exe  :::  756
svchost.exe  :::  832
svchost.exe  :::  912
winlogon.exe  :::  920
fontdrvhost.exe  :::  976
svchost.exe  :::  1040
svchost.exe  :::  1052
svchost.exe  :::  1060
svchost.exe  :::  1096
svchost.exe  :::  1120
svchost.exe  :::  1128
svchost.exe  :::  1152
svchost.exe  :::  1176
svchost.exe  :::  1188
svchost.exe  :::  1260
ShellExperienceHost.exe  :::  1352
svchost.exe  :::  1364
svchost.exe  :::  1408
svchost.exe  :::  1492
chrome.exe  :::  1520
svchost.exe  :::  1528
svchost.exe  :::  1548
MicrosoftEdgeCP.exe  :::  1612
svchost.exe  :::  1688
atiesrxx.exe  :::  1708
svchost.exe  :::  1728
sv

In [ ]:
import subprocess
def get_pname(id):
    p = subprocess.Popen(["ps -o cmd= {}".format(id)], stdout=subprocess.PIPE, shell=True)
    return str(p.communicate()[0])
name = get_pname(1)
print(name)

In [ ]:
    cmd = 'powershell "gps | where {$_.MainWindowTitle } | select Description, id'
#abrimos la consola y ejecutamos CMD    
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    #Iteramos el resultado de la consola.
    for line in proc.stdout:
        #Como devuelve Bytes, lo codificamos a str. y buscamos el proceso 'Excel'.
        if line.decode(errors='ignore').find('Excel') != -1: # <-------------------- Linea (errors='ignore) agregada por jose 11/03/2020
            #Si lo encuentra significa que es el candidato a morir,
            #Entonces tomamos la linea y hacemos un split() para obtener el ultimo registro.
            #Lo hacemos porque un ejemplo de output puede ser:
            #'Microsoft Office Excel                                                3512'
            #Entonces nos aseguramos que [-1] Siempre va a ser el id si lo cortamos por espacios en blanco.
            candidato = line.decode().split()[-1]
            #Ejecuta la función kill de sistema operativo, con el id del candidato a matar, con la signal.SIGTERM.
            os.kill(int(candidato), signal.SIGTERM)

In [21]:
for i in df_errores[3].unique():
    print("---"*25)
    print(i)

---------------------------------------------------------------------------
13188
---------------------------------------------------------------------------
13354
---------------------------------------------------------------------------
14198
---------------------------------------------------------------------------
14199
---------------------------------------------------------------------------
14200
---------------------------------------------------------------------------
14201
---------------------------------------------------------------------------
14202
---------------------------------------------------------------------------
14203
---------------------------------------------------------------------------
14204
---------------------------------------------------------------------------
14205
---------------------------------------------------------------------------
14206
---------------------------------------------------------------------------
14207
----------------

---------------------------------------------------------------------------
15241
---------------------------------------------------------------------------
15242
---------------------------------------------------------------------------
15243
---------------------------------------------------------------------------
15244
---------------------------------------------------------------------------
15245
---------------------------------------------------------------------------
15246
---------------------------------------------------------------------------
15247
---------------------------------------------------------------------------
15248
---------------------------------------------------------------------------
15249
---------------------------------------------------------------------------
15250
---------------------------------------------------------------------------
15251
---------------------------------------------------------------------------
15252
----------------

---------------------------------------------------------------------------
17968
---------------------------------------------------------------------------
18062
---------------------------------------------------------------------------
18063
---------------------------------------------------------------------------
18064
---------------------------------------------------------------------------
18065
---------------------------------------------------------------------------
18066
---------------------------------------------------------------------------
18067
---------------------------------------------------------------------------
18068
---------------------------------------------------------------------------
18069
---------------------------------------------------------------------------
18070
---------------------------------------------------------------------------
18071
---------------------------------------------------------------------------
18072
----------------

In [16]:
list(df_errores[2].unique())

["(com_error(-2147352567, 'Ocurrió una excepción.', (0, 'Microsoft Office Excel', 'Error en el método Open de la clase Workbooks.', 'C:\\\\Program Files (x86)\\\\Microsoft Office\\\\Office12\\\\3082\\\\XLMAIN11.CHM', 0, -2146827284), None), [])",
 '(com_error(-2147352567, \'Ocurrió una excepción.\', (0, \'Microsoft Office Excel\', "No se puede obtener acceso al archivo \'Z:\\\\Planeamiento\\\\Archivos varios Ale\\\\11- Indice-Catalogo\\\\INDICE 1901.xlsx\'. Puede haber varios motivos:\\n\\n• El nombre del archivo o la ruta no existen.\\n• Otro programa está usando el archivo.\\n• El libro que está intentando guardar tiene el mismo nombre que otro libro que está abierto en estos momentos.", \'C:\\\\Program Files (x86)\\\\Microsoft Office\\\\Office12\\\\3082\\\\XLMAIN11.CHM\', 0, -2146827284), None), [])',
 '(com_error(-2147352567, \'Ocurrió una excepción.\', (0, \'Microsoft Office Excel\', "No se puede obtener acceso al archivo \'Z:\\\\Planeamiento\\\\Archivos varios Ale\\\\11- Indice-C

In [22]:
len(df_errores) + len(df_output)

2076

In [23]:
len(df_errores)

2059

In [24]:
len(df_output)

17

In [25]:
df_output

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,Consulta desde estimaciones,"select 0 id, null fecha, null cod8, null insum...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,id,$A$16,$D$9,Calculo,732,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.449391
1,1,Consulta desde estimaciones,"select 0 id, null fecha, null cod8, null insum...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,id,$A$16,$D$10,Calculo,04009,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.501447
2,2,Consulta desde estimaciones,"select 0 id, null fecha, null cod8, null insum...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,id,$A$16,$D$8,Calculo,67513340,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.548293
3,3,Consulta desde estimaciones,"select 0 id, null fecha, null cod8, null insum...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,id,$A$16,$D$11,Calculo,"0,90",33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.595139
4,4,Consulta desde estimaciones1,"select i.codigo, color, descrip, precio , rind...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,codigo,$CK$9,$D$9,Calculo,732,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.813951
5,5,Consulta desde estimaciones1,"select i.codigo, color, descrip, precio , rind...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,codigo,$CK$9,$D$10,Calculo,04009,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:11.860825
6,6,Consulta desde estimaciones2,"select \r\nsubstring(f.articulo,1,8) cod8, sub...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,cod8,$CK$16,$D$9,Calculo,732,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:12.005044
7,7,Consulta desde estimaciones2,"select \r\nsubstring(f.articulo,1,8) cod8, sub...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,cod8,$CK$16,$D$10,Calculo,04009,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:12.130045
8,8,Consulta desde estimaciones2,"select \r\nsubstring(f.articulo,1,8) cod8, sub...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,cod8,$CK$16,$D$8,Calculo,67513340,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:12.176921
9,9,Consulta desde estimaciones23,"select \r\nsubstring(f.articulo,1,8) cod8, sub...",ODBC;DSN=estimaciones;Description=estimaciones...,Calculo,cod8,$CK$30,$D$9,Calculo,732,33570,19100,2020-03-16 08:39:07.641762,2020-03-16 08:39:12.270672


In [49]:
pd.set_option('display.max_columns', False)

In [50]:
df_output = pd.read_excel("temp_output.xlsx")
df_errores = pd.read_excel("temp_errores.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'temp_output.xlsx'

In [61]:
len(df_output)

5484

In [89]:
len(df_errores)

1359

In [40]:
df_errores

NameError: name 'df_errores' is not defined

In [ ]:
#12600 antes del fin de semana
#19100 luego del fin de semana

In [88]:
len(df_output)

5484

In [96]:
len(df_errores) + len(df_output)

15667

In [1]:
import pandas as pd
hoja1 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_output.xlsx")
hoja2 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_errores.xlsx")

hoja3 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_output_ultimo.xlsx")
hoja4 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_errores_ultimo.xlsx")

hoja5 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_output_12_03.xlsx")
hoja6 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_errores_13-03.xlsx")

hoja7 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_output_16-03.xlsx")
hoja8 =  pd.read_excel("Z:\\Planeamiento\\Archivos varios Lucas\\Relevamiento_excel\\temp_errores_16-03.xlsx")

In [2]:
vacio =  pd.DataFrame()
outputs = list(map(vacio.append,[hoja1,hoja3,hoja5,hoja7]))
errores = list(map(vacio.append,[hoja2,hoja4,hoja6,hoja8]))

In [53]:
hoja_completa

[      Unnamed: 0  ...                          12
 0              0  ...  2020-03-06 12:54:38.757119
 1              1  ...  2020-03-06 12:54:38.805121
 2              2  ...  2020-03-06 12:54:43.413231
 3              3  ...  2020-03-06 12:54:43.475733
 4              4  ...  2020-03-06 12:54:43.538234
 ...          ...  ...                         ...
 9814        9814  ...  2020-03-07 02:18:56.528147
 9815        9815  ...  2020-03-07 02:18:56.575022
 9816        9816  ...  2020-03-07 02:18:56.637491
 9817        9817  ...  2020-03-07 02:18:56.715647
 9818        9818  ...  2020-03-07 02:18:56.778148
 
 [9819 rows x 14 columns],        Unnamed: 0  ...      3
 0               0  ...      1
 1               1  ...      3
 2               2  ...      7
 3               3  ...      8
 4               4  ...      9
 ...           ...  ...    ...
 15508       15508  ...  17146
 15509       15509  ...  17147
 15510       15510  ...  17148
 15511       15511  ...  17149
 15512       15512 

In [44]:
output_final  = pd.concat(outputs)
errores_final = pd.concat(errores)

In [10]:
len(output_final)

23646

In [35]:
len(errores_final)

40061

In [36]:
len(errores_final) + len(output_final)

63707

In [45]:
errores_final[2]

0                          ('No se pudo abrir .xlsx', [])
1                          ('No se pudo abrir .xlsx', [])
2                          ('No se pudo abrir .xlsx', [])
3                          ('No se pudo abrir .xlsx', [])
4                          ('No se pudo abrir .xlsx', [])
                              ...                        
2286    (com_error(-2147352567, 'Ocurrió una excepción...
2287    (com_error(-2147352567, 'Ocurrió una excepción...
2288    (com_error(-2147352567, 'Ocurrió una excepción...
2289    (com_error(-2147352567, 'Ocurrió una excepción...
2290    (com_error(-2147352567, 'Ocurrió una excepción...
Name: 2, Length: 40061, dtype: object

In [23]:
errores_final.drop_duplicates(subset=['2'])

KeyError: Index(['2'], dtype='object')

In [27]:
result_df = errores_final.drop_duplicates(keep='2')

ValueError: keep must be either "first", "last" or False

In [82]:
result_df.iloc[0][0]

41

In [4]:
outputs

[      Unnamed: 0                              0  \
 0              0    Consulta desde ESTIMACIONES   
 1              1   Consulta desde ESTIMACIONES1   
 2              2    Consulta desde ESTIMACIONES   
 3              3   Consulta desde ESTIMACIONES1   
 4              4   Consulta desde estimaciones2   
 ...          ...                            ...   
 9814        9814  Consulta desde estimaciones11   
 9815        9815  Consulta desde estimaciones12   
 9816        9816   Consulta desde estimaciones4   
 9817        9817  Consulta desde estimaciones41   
 9818        9818   Consulta desde estimaciones5   
 
                                                       1  \
 0     SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...   
 1     SELECT colores.codigo, colores.nombre\r\nFROM ...   
 2     SELECT f.articulo, f.insumo, f.color, i.descri...   
 3     SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...   
 4     SELECT taller.codigo, taller.nombre, taller.do...   
 ...          

In [5]:
errores

[       Unnamed: 0      0                                                  1  \
 0               0     41  Z:\....Candidatos a borrar\Archivos Viejos\Obs...   
 1               1     43  Z:\....Candidatos a borrar\Carpetas compartida...   
 2               2     47  Z:\....Candidatos a borrar\varios\javier zonas...   
 3               3     48  Z:\....Candidatos a borrar\varios\lista campañ...   
 4               4     49  Z:\....Candidatos a borrar\varios\lista campañ...   
 ...           ...    ...                                                ...   
 15508       15508  17186  Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...   
 15509       15509  17187  Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...   
 15510       15510  17188  Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...   
 15511       15511  17189  Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...   
 15512       15512  17190  Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...   
 
                                      